In [1]:
import akshare as ak
import pandas as pd
import time

def fetch_clean_and_merge_bonds():
    """
    获取、清洗、比对并合并东方财富和同花顺的历史可转债数据，
    最终将结果保存到一个结构清晰的Excel文件中。
    """
    print("🚀 任务启动：开始获取、清洗并合并历史可转债数据...")

    # --- 步骤 1: 从东方财富和同花顺获取可转债列表 ---
    try:
        print("\n[步骤 1/5] 正在从东方财富获取可转债列表...")
        em_bonds_df = ak.bond_zh_cov()
        print(f"  ✅ 从东方财富成功获取 {len(em_bonds_df)} 条记录。")
    except Exception as e:
        print(f"  ❌ 从东方财富获取数据失败，原因: {e}")
        return

    # 添加短暂延时，避免请求过于频繁
    time.sleep(2) 

    try:
        print("\n[步骤 1/5] 正在从同花顺获取可转债列表...")
        ths_bonds_df = ak.bond_zh_cov_info_ths()
        print(f"  ✅ 从同花顺成功获取 {len(ths_bonds_df)} 条记录。")
    except Exception as e:
        print(f"  ❌ 从同花顺获取数据失败，原因: {e}")
        return

    # --- 步骤 2: 清洗东方财富数据，确保品种纯粹性 ---
    print("\n[步骤 2/5] 正在清洗东方财富数据，剔除非可转债品种...")
    original_em_count = len(em_bonds_df)
    
    # 根据我们的分析，东方财富的数据包含了一些非可转债品种。
    # 我们通过筛选债券简称中是否包含“转债”或“退债”来过滤，以保证数据纯度。
    # '退债' 是为了保留那些已经退市的可转债记录。
    filter_condition = em_bonds_df['债券简称'].str.contains('转债|退债', na=False)
    em_bonds_df_cleaned = em_bonds_df[filter_condition].copy()
    
    cleaned_em_count = len(em_bonds_df_cleaned)
    removed_count = original_em_count - cleaned_em_count
    print(f"  ✅ 清洗完成。从东方财富数据中移除了 {removed_count} 条非可转债记录，保留 {cleaned_em_count} 条。")

    # --- 步骤 3: 数据合并与差异识别 ---
    print("\n[步骤 3/5] 正在合并两个数据源并识别差异...")
    
    # 使用 'outer' 方式合并，确保两个列表中的所有记录都被保留。
    # 'indicator=True' 参数会自动生成一个 '_merge' 列，标记每条记录的来源。
    # 'suffixes' 用于处理两个数据源中重名的列，如“债券简称”。
    merged_df = pd.merge(
        em_bonds_df_cleaned,
        ths_bonds_df,
        on='债券代码',
        how='outer',
        suffixes=('_em', '_ths'),
        indicator=True
    )

    # 将自动生成的来源标识列 '_merge' 转换为更具可读性的中文说明。
    source_map = {
        'both': '共同存在',
        'left_only': '仅东方财富',
        'right_only': '仅同花顺'
    }
    merged_df['数据来源'] = merged_df['_merge'].map(source_map)
    merged_df.drop('_merge', axis=1, inplace=True)
    
    # 统计并打印差异信息
    source_counts = merged_df['数据来源'].value_counts()
    print("  ✅ 数据合并完成，来源分布如下:")
    print(f"    - 两个平台共同拥有的债券数量: {source_counts.get('共同存在', 0)}")
    print(f"    - 仅在东方财富(清洗后)中存在的债券数量: {source_counts.get('仅东方财富', 0)}")
    print(f"    - 仅在同花顺中存在的债券数量: {source_counts.get('仅同花顺', 0)}")

    # --- 步骤 4: 整理与优化DataFrame的列顺序 ---
    print("\n[步骤 4/5] 正在整理列顺序以便于查看...")
    
    # 将关键信息和标识列提前，方便快速浏览和筛选。
    preferred_columns = [
        '债券代码',
        '数据来源',
        '债券简称_em',
        '债券简称_ths',
        '正股代码_em',
        '正股简称_em',
        '正股代码_ths',
        '正股简称_ths',
        '上市时间', # from em
        '上市日期',   # from ths
    ]
    
    # 获取所有其他列，并从其他列中移除我们已经指定的优先列
    other_columns = [col for col in merged_df.columns if col not in preferred_columns]
    
    # 对剩余列进行排序，确保输出的表格结构稳定且易于查找
    final_column_order = preferred_columns + sorted(other_columns)
    
    # 应用新的列顺序
    final_df = merged_df[final_column_order]
    print("  ✅ 列顺序调整完毕。")

    # --- 步骤 5: 导出到Excel文件 ---
    output_filename = "历史可转债数据全集(已清洗和合并).xlsx"
    print(f"\n[步骤 5/5] 正在将结果导出到Excel文件: {output_filename} ...")
    try:
        with pd.ExcelWriter(output_filename, engine='openpyxl') as writer:
            final_df.to_excel(writer, index=False, sheet_name='可转债全集')
        print(f"  🎉 任务成功完成！数据已保存至您的当前目录下的 '{output_filename}' 文件中。")
    except Exception as e:
        print(f"  ❌ 导出Excel失败，原因: {e}")

if __name__ == '__main__':
    # 执行主函数
    fetch_clean_and_merge_bonds()

🚀 任务启动：开始获取、清洗并合并历史可转债数据...

[步骤 1/5] 正在从东方财富获取可转债列表...


  0%|          | 0/2 [00:00<?, ?it/s]

c:\Program Files\Python313\Lib\site-packages\akshare\bond\bond_zh_cov.py:342: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  big_df = pd.concat(objs=[big_df, temp_df], ignore_index=True)


  ✅ 从东方财富成功获取 983 条记录。

[步骤 1/5] 正在从同花顺获取可转债列表...
  ✅ 从同花顺成功获取 890 条记录。

[步骤 2/5] 正在清洗东方财富数据，剔除非可转债品种...
  ✅ 清洗完成。从东方财富数据中移除了 55 条非可转债记录，保留 928 条。

[步骤 3/5] 正在合并两个数据源并识别差异...
  ✅ 数据合并完成，来源分布如下:
    - 两个平台共同拥有的债券数量: 850
    - 仅在东方财富(清洗后)中存在的债券数量: 78
    - 仅在同花顺中存在的债券数量: 40

[步骤 4/5] 正在整理列顺序以便于查看...
  ✅ 列顺序调整完毕。

[步骤 5/5] 正在将结果导出到Excel文件: 历史可转债数据全集(已清洗和合并).xlsx ...
  🎉 任务成功完成！数据已保存至您的当前目录下的 '历史可转债数据全集(已清洗和合并).xlsx' 文件中。
